In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn import svm
import joblib

In [ ]:
def load_and_preprocess_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert('L')  # Convert to grayscale
            img = img.resize((64, 64))  # Resize to 64×64 pixels
            img_array = np.array(img)
            img_array_fl = img_array.flatten()  # Flatten the 2D image to 1D for SVM
            images.append(img_array_fl)
            labels.append(label)  # Label for the category
    return np.array(images), np.array(labels)

In [ ]:
# prompt: data from drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
categories = {
    "With_mask": "/content/drive/MyDrive/Newdataset/Newdataset/With_mask",
    "Without_mask": "/content/drive/MyDrive/Newdataset/Newdataset/Without_mask",
    "Incorrect_mask": "/content/drive/MyDrive/Newdataset/Newdataset/Incorrect_mask"
}

In [ ]:
# Load images and labels for all categories
all_images = []
all_labels = []
for label, folder_path in enumerate(categories.values()):
    images, labels = load_and_preprocess_images(folder_path, label)
    all_images.append(images)
    all_labels.append(labels)

In [ ]:
# Combine all data
all_images = np.vstack(all_images)
all_labels = np.concatenate(all_labels)

# Normalize images
all_images = all_images / 255.0

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.1, random_state=42)

# Create an SVM classifier
clf = svm.SVC(gamma='scale')

In [ ]:
# Train the SVM classifier
clf.fit(X_train, y_train)

SVC()

In [ ]:
# Evaluate the model
accuracy = clf.score(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 90.33%


In [ ]:
# Save the trained model
joblib_file = "multi_class_svm_model.pkl"
joblib.dump(clf, joblib_file)
print(f"Model saved to {joblib_file}")


Model saved to multi_class_svm_model.pkl


In [ ]:
def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    img = img.resize((64, 64))  # Resize to match training images
    img_array = np.array(img)
    img_array = img_array.flatten()  # Flatten the image
    img_array = img_array / 255.0  # Normalize the image
    return img_array

def predict_category(image_path, model, categories):
    preprocessed_image = preprocess_image(image_path)
    preprocessed_image = preprocessed_image.reshape(1, -1)  # Reshape for the model
    prediction = model.predict(preprocessed_image)[0]
    category = list(categories.keys())[prediction]
    return category

# Example usage
new_image_path = '/content/drive/MyDrive/Newdataset/Newdataset/With_mask/simple100.jpg'  # Path to the new image
model = joblib.load("multi_class_svm_model.pkl")  # Load the saved model
predicted_category = predict_category(new_image_path, model, categories)
print(f"The image most likely belongs to: {predicted_category}")

The image most likely belongs to: With_mask
